<img src="https://cdn-images-1.medium.com/max/2000/1*LLVL8xUiUOBE8WHgzAuY-Q.png" width="600px">

# Pytorch Intro | MNIST Classification

In [1]:
import torch
torch.manual_seed(1)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Data and Dataloader
<img src="https://cdn-images-1.medium.com/max/1200/1*9Mjoc_J0JR294YwHGXwCeg.jpeg" width="300px">

In [2]:
from torchvision import datasets, transforms
data_train = datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

data_test = datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

In [3]:
print(data_train.data.shape)
print(data_train.targets.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])


In [4]:
train_loader = torch.utils.data.DataLoader(data_train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=64, shuffle=True)

## Model

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
model = Net()

In [7]:
model

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [8]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0272, -0.0286, -0.0184,  ...,  0.0236, -0.0228,  0.0153],
        [-0.0234, -0.0054, -0.0145,  ...,  0.0223, -0.0192,  0.0007],
        [ 0.0079, -0.0050, -0.0150,  ...,  0.0159,  0.0159,  0.0237],
        ...,
        [-0.0106,  0.0098,  0.0071,  ...,  0.0114, -0.0208,  0.0233],
        [-0.0002,  0.0225, -0.0062,  ..., -0.0230,  0.0125,  0.0288],
        [-0.0125,  0.0301, -0.0215,  ..., -0.0144,  0.0352, -0.0332]],
       requires_grad=True)

In [9]:
model.to(device)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [10]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0272, -0.0286, -0.0184,  ...,  0.0236, -0.0228,  0.0153],
        [-0.0234, -0.0054, -0.0145,  ...,  0.0223, -0.0192,  0.0007],
        [ 0.0079, -0.0050, -0.0150,  ...,  0.0159,  0.0159,  0.0237],
        ...,
        [-0.0106,  0.0098,  0.0071,  ...,  0.0114, -0.0208,  0.0233],
        [-0.0002,  0.0225, -0.0062,  ..., -0.0230,  0.0125,  0.0288],
        [-0.0125,  0.0301, -0.0215,  ..., -0.0144,  0.0352, -0.0332]],
       device='cuda:0', requires_grad=True)

## Optimizer

In [11]:
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Train schedule

In [12]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) # The same as model.forward(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [13]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Run

In [ ]:
for epoch in range(1, 100 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.716533
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.555295
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.444822
